In [1]:
# Names of datasets to be used
# '/Users/darionguan/Desktop/Presidential Prediction/acs_state_final.csv'
# '/Users/darionguan/Desktop/Presidential Prediction/acs_cd_final.csv'
# '/Users/darionguan/Desktop/Presidential Prediction/biden.csv'
# '/Users/darionguan/Desktop/Presidential Prediction/trump.csv'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import pandasql as psql

In [3]:
filename = '/Users/darionguan/Desktop/Presidential Prediction/acs_state_final.csv'
acs_df = pd.read_csv(filename).iloc[: , 1:]
acs_df['FIPS'] = acs_df['FIPS'].apply('{:0>2}'.format)
acs_df.head()

,State,STATE_ABB,FIPS,MedianHouseholdIncome,MedianNonfamilyIncome,Pct18to64InPoverty,PctOver65InPoverty,PctUnemployedCivilians,PctLessThan9thGrade,Pct9thto12thGrade,PctHSGrad,PctSomeCollege,PctAssociatesDeg,PctBachelorsDeg,PctGradDeg
0,Alabama,AL,01,51734,29458,0.147,0.105,0.049,0.039,0.090,0.309,0.208,0.090,0.163,0.100
1,Alaska,AK,02,75463,47508,0.095,0.069,0.058,0.022,0.043,0.287,0.257,0.090,0.185,0.117
2,Arizona,AZ,04,62055,40198,0.127,0.090,0.051,0.051,0.073,0.237,0.250,0.087,0.188,0.113
3,Arkansas,AR,05,48952,27607,0.155,0.105,0.048,0.046,0.079,0.349,0.218,0.075,0.151,0.083
4,California,CA,06,80440,51676,0.107,0.105,0.051,0.087,0.073,0.206,0.206,0.079,0.219,0.131


In [4]:
state_abb_dict = {'Alabama': 'AL',
                  'Alaska': 'AK',
                  'Arizona': 'AZ',
                  'Arkansas': 'AR',
                  'California': 'CA',
                  'Colorado': 'CO',
                  'Connecticut': 'CT',
                  'Delaware': 'DE',
                  'DC': 'DC',
                  'Florida': 'FL',
                  'Georgia': 'GA',
                  'Hawaii': 'HI',
                  'Idaho': 'ID',
                  'Illinois': 'IL',
                  'Indiana': 'IN',
                  'Iowa': 'IA',
                  'Kansas': 'KS',
                  'Kentucky': 'KY',
                  'Louisiana': 'LA',
                  'Maine': 'ME',
                  'Maryland': 'MD',
                  'Massachusetts': 'MA',
                  'Michigan': 'MI',
                  'Minnesota': 'MN',
                  'Mississippi': 'MS',
                  'Missouri': 'MO',
                  'Montana': 'MT',
                  'Nebraska': 'NE',
                  'Nevada': 'NV',
                  'NewHampshire': 'NH',
                  'NewJersey': 'NJ',
                  'NewMexico': 'NM',
                  'NewYork': 'NY',
                  'NorthCarolina': 'NC',
                  'NorthDakota': 'ND',
                  'Ohio': 'OH',
                  'Oklahoma': 'OK',
                  'Oregon': 'OR',
                  'Pennsylvania': 'PA',
                  'RhodeIsland': 'RI',
                  'SouthCarolina': 'SC',
                  'SouthDakota': 'SD',
                  'Tennessee': 'TN',
                  'Texas': 'TX',
                  'Utah': 'UT',
                  'Vermont': 'VT',
                  'Virginia': 'VA',
                  'Washington': 'WA',
                  'WestVirginia': 'WV',
                  'Wisconsin': 'WI',
                  'Wyoming': 'WY'}

In [5]:
filename = '/Users/darionguan/Desktop/Presidential Prediction/polling/president_polls.csv'
biden_df = pd.read_csv(filename)
biden_df = biden_df[['question_id', 'poll_id', 'state', 'fte_grade', 'sample_size', 'start_date', 'end_date',
               'created_at', 'answer', 'candidate_id', 'pct']]
biden_df = biden_df[(biden_df['candidate_id'] == 13256)]
biden_df = biden_df[(biden_df['fte_grade'] == 'A+') | (biden_df['fte_grade'] == 'A') |
                            (biden_df['fte_grade'] == 'A-') | (biden_df['fte_grade'] == 'A/B') |
                            (biden_df['fte_grade'] == 'B+') | (biden_df['fte_grade'] == 'B') |
                            (biden_df['fte_grade'] == 'B-') | (biden_df['fte_grade'] == 'B/C') |
                            (biden_df['fte_grade'] == 'C+') | (biden_df['fte_grade'] == 'C')]
# biden_df['ZIP_CODE'] = biden_df['ZIP_CODE'].apply(np.int64)
biden_df['sample_size'] = biden_df['sample_size'].apply(np.int64)
biden_df['start_date'] = pd.to_datetime(biden_df['start_date'])
biden_df['end_date'] = pd.to_datetime(biden_df['end_date'])
biden_df['created_at'] = pd.to_datetime(biden_df['created_at'])
biden_df = biden_df.dropna(subset=['state'])
biden_df['state'] = biden_df.state.mask(biden_df.state.str.contains('CD'), biden_df.state.str[0:-5])
biden_df['state'] = biden_df['state'].str.replace(' ', '')
# acs_df.insert(0, 'STATE_ABB', acs_df['State'].map(state_abb_dict))
biden_df = biden_df.replace({'state': state_abb_dict})
#######
biden_df = biden_df.groupby('state')['pct'].mean().to_frame().reset_index()
biden_df = biden_df.reset_index(drop=True)

q = """
SELECT b.state,
        b.pct,
        a.MedianHouseholdIncome,
        a.MedianNonfamilyIncome,
        a.Pct18to64InPoverty,
        a.PctOver65InPoverty,
        a.PctUnemployedCivilians,
        a.PctLessThan9thGrade,
        a.Pct9thto12thGrade,
        a.PctHSGrad,
        a.PctSomeCollege,
        a.PctAssociatesDeg,
        a.PctBachelorsDeg,
        a.PctGradDeg
FROM biden_df AS b
LEFT JOIN acs_df AS a
    ON b.state = a.STATE_ABB
"""

biden_df = psql.sqldf(q)

biden_df = biden_df.sort_values('state')

# biden_df = biden_df.reset_index(drop=True)

biden_df.to_csv('biden_polls.csv')
biden_df

/var/folders/k1/z3v9vvcx2zj8knzj63hcxqh40000gn/T/ipykernel_46838/609361842.py:2: DtypeWarning: Columns (6,22,25,29) have mixed types. Specify dtype option on import or set low_memory=False.
  biden_df = pd.read_csv(filename)


,state,pct,MedianHouseholdIncome,MedianNonfamilyIncome,Pct18to64InPoverty,PctOver65InPoverty,PctUnemployedCivilians,PctLessThan9thGrade,Pct9thto12thGrade,PctHSGrad,PctSomeCollege,PctAssociatesDeg,PctBachelorsDeg,PctGradDeg
0,AK,43.955556,75463,47508,0.095,0.069,0.058,0.022,0.043,0.287,0.257,0.090,0.185,0.117
1,AL,38.714286,51734,29458,0.147,0.105,0.049,0.039,0.090,0.309,0.208,0.090,0.163,0.100
2,AR,36.833333,48952,27607,0.155,0.105,0.048,0.046,0.079,0.349,0.218,0.075,0.151,0.083
3,AZ,48.257573,62055,40198,0.127,0.090,0.051,0.051,0.073,0.237,0.250,0.087,0.188,0.113
4,CA,59.256429,80440,51676,0.107,0.105,0.051,0.087,0.073,0.206,0.206,0.079,0.219,0.131
5,CO,52.529167,77127,50092,0.093,0.072,0.037,0.032,0.044,0.210,0.203,0.084,0.266,0.160
6,CT,51.785714,78833,45001,0.094,0.073,0.053,0.040,0.053,0.268,0.165,0.076,0.220,0.178
7,DE,56.950000,70176,44071,0.108,0.073,0.046,0.035,0.062,0.302,0.186,0.083,0.195,0.137
8,FL,48.704741,59227,37292,0.117,0.107,0.045,0.046,0.070,0.284,0.194,0.099,0.193,0.114
9,GA,47.058630,61980,37881,0.119,0.104,0.048,0.045,0.076,0.274,0.200,0.080,0.199,0.126


In [6]:
filename = '/Users/darionguan/Desktop/Presidential Prediction/polling/president_polls.csv'
trump_df = pd.read_csv(filename)
trump_df = trump_df[['question_id', 'poll_id', 'state', 'fte_grade', 'sample_size', 'start_date', 'end_date',
               'created_at', 'answer', 'candidate_id', 'pct']]
trump_df = trump_df[(trump_df['candidate_id'] == 13254)]
trump_df = trump_df[(trump_df['fte_grade'] == 'A+') | (trump_df['fte_grade'] == 'A') |
                            (trump_df['fte_grade'] == 'A-') | (trump_df['fte_grade'] == 'A/B') |
                            (trump_df['fte_grade'] == 'B+') | (trump_df['fte_grade'] == 'B') |
                            (trump_df['fte_grade'] == 'B-') | (trump_df['fte_grade'] == 'B/C') |
                            (trump_df['fte_grade'] == 'C+') | (trump_df['fte_grade'] == 'C')]
# trump_df['ZIP_CODE'] = trump_df['ZIP_CODE'].apply(np.int64)
trump_df['sample_size'] = trump_df['sample_size'].apply(np.int64)
trump_df['start_date'] = pd.to_datetime(trump_df['start_date'])
trump_df['end_date'] = pd.to_datetime(trump_df['end_date'])
trump_df['created_at'] = pd.to_datetime(trump_df['created_at'])
trump_df = trump_df.dropna(subset=['state'])
trump_df['state'] = trump_df.state.mask(trump_df.state.str.contains('CD'), trump_df.state.str[0:-5])
trump_df['state'] = trump_df['state'].str.replace(' ', '')
# acs_df.insert(0, 'STATE_ABB', acs_df['State'].map(state_abb_dict))
trump_df = trump_df.replace({'state': state_abb_dict})
trump_df = trump_df.groupby('state')['pct'].mean().to_frame().reset_index()
trump_df = trump_df.reset_index(drop=True)


q = """
SELECT t.state,
        t.pct,
        a.MedianHouseholdIncome,
        a.MedianNonfamilyIncome,
        a.Pct18to64InPoverty,
        a.PctOver65InPoverty,
        a.PctUnemployedCivilians,
        a.PctLessThan9thGrade,
        a.Pct9thto12thGrade,
        a.PctHSGrad,
        a.PctSomeCollege,
        a.PctAssociatesDeg,
        a.PctBachelorsDeg,
        a.PctGradDeg
FROM trump_df AS t
LEFT JOIN acs_df AS a
    ON t.state = a.STATE_ABB
"""

trump_df = psql.sqldf(q)
trump_df.to_csv('trump_polls.csv')
trump_df

/var/folders/k1/z3v9vvcx2zj8knzj63hcxqh40000gn/T/ipykernel_46838/3612912599.py:2: DtypeWarning: Columns (6,22,25,29) have mixed types. Specify dtype option on import or set low_memory=False.
  trump_df = pd.read_csv(filename)


,state,pct,MedianHouseholdIncome,MedianNonfamilyIncome,Pct18to64InPoverty,PctOver65InPoverty,PctUnemployedCivilians,PctLessThan9thGrade,Pct9thto12thGrade,PctHSGrad,PctSomeCollege,PctAssociatesDeg,PctBachelorsDeg,PctGradDeg
0,AK,47.830769,75463,47508,0.095,0.069,0.058,0.022,0.043,0.287,0.257,0.090,0.185,0.117
1,AL,56.833333,51734,29458,0.147,0.105,0.049,0.039,0.090,0.309,0.208,0.090,0.163,0.100
2,AR,56.666667,48952,27607,0.155,0.105,0.048,0.046,0.079,0.349,0.218,0.075,0.151,0.083
3,AZ,45.642574,62055,40198,0.127,0.090,0.051,0.051,0.073,0.237,0.250,0.087,0.188,0.113
4,CA,33.500633,80440,51676,0.107,0.105,0.051,0.087,0.073,0.206,0.206,0.079,0.219,0.131
5,CO,41.339286,77127,50092,0.093,0.072,0.037,0.032,0.044,0.210,0.203,0.084,0.266,0.160
6,CT,34.018750,78833,45001,0.094,0.073,0.053,0.040,0.053,0.268,0.165,0.076,0.220,0.178
7,DE,43.600000,70176,44071,0.108,0.073,0.046,0.035,0.062,0.302,0.186,0.083,0.195,0.137
8,FL,46.317285,59227,37292,0.117,0.107,0.045,0.046,0.070,0.284,0.194,0.099,0.193,0.114
9,GA,46.704457,61980,37881,0.119,0.104,0.048,0.045,0.076,0.274,0.200,0.080,0.199,0.126
